In [1]:
#generate embeddings for mutli-organ segmentation with present multi-organ mask

In [4]:
import os
import clip
import torch
import pickle


## PAOT
ORGAN_NAME = ['Spleen', 'Right Kidney', 'Left Kidney', 'Gall Bladder', 'Esophagus', 
                'Liver', 'Stomach', 'Arota', 'Postcava', 'Portal Vein and Splenic Vein',
                'Pancreas', 'Right Adrenal Gland', 'Left Adrenal Gland', 'Duodenum', 'Hepatic Vessel',
                'Right Lung', 'Left Lung', 'Colon', 'Intestine', 'Rectum', 
                'Bladder', 'Prostate', 'Left Head of Femur', 'Right Head of Femur', 'Celiac Truck',
                'Kidney Tumor', 'Liver Tumor', 'Pancreas Tumor', 'Hepatic Vessel Tumor', 'Lung Tumor', 
                'Colon Tumor', 'Kidney Cyst']

TEMPLATE={
    '01': [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    '02': [11],
    '03': [6],
    '04': [6,27], # post process
    '05': [2,3,26,32], # post process
    '06': [1,2,3,4,6,7,11,16,17],
    '07': [6,1,3,2,7,4,5,11,14,18,19,12,13,20,21,23,24],
    '08': [6, 2, 3, 1, 11],
    '09': [1,2,3,4,5,6,7,8,9,11,12,13,14,21,22],
    '12': [6,21,16,17,2,3],  
    '13': [6,2,3,1,11,8,9,7,4,5,12,13,25], 
    '14': [11, 28],
    '10_03': [6, 27], # post process
    '10_06': [30],
    '10_07': [11, 28], # post process
    '10_08': [15, 29], # post process
    '10_09': [1],
    '10_10': [31],
    #'15': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] ## total segmentation
}

In [5]:
#calculate according to the template the clip embeddings and save. 

In [8]:
# Function to generate sentence
def generate_sentence(template):
    selected_organs = []
    for i in template:
        selected_organs.append(ORGAN_NAME[i-1])
    return "a computerized tomography of " + ", ".join(selected_organs)

In [14]:
def generate_sentence_list(template):
    selected_organs = []
    for i in template:
        selected_organs.append(f"a computerized tomography of {ORGAN_NAME[i-1]}")
    return selected_organs

In [33]:
template_text = {}
for key, value in TEMPLATE.items():
    template_text[key] = generate_sentence(value)

In [34]:
template_text

{'01': 'a computerized tomography of Spleen, Right Kidney, Left Kidney, Gall Bladder, Esophagus, Liver, Stomach, Arota, Postcava, Portal Vein and Splenic Vein, Pancreas, Right Adrenal Gland, Left Adrenal Gland, Duodenum',
 '02': 'a computerized tomography of Pancreas',
 '03': 'a computerized tomography of Liver',
 '04': 'a computerized tomography of Liver, Liver Tumor',
 '05': 'a computerized tomography of Right Kidney, Left Kidney, Kidney Tumor, Kidney Cyst',
 '06': 'a computerized tomography of Spleen, Right Kidney, Left Kidney, Gall Bladder, Liver, Stomach, Pancreas, Right Lung, Left Lung',
 '07': 'a computerized tomography of Liver, Spleen, Left Kidney, Right Kidney, Stomach, Gall Bladder, Esophagus, Pancreas, Duodenum, Colon, Intestine, Right Adrenal Gland, Left Adrenal Gland, Rectum, Bladder, Left Head of Femur, Right Head of Femur',
 '08': 'a computerized tomography of Liver, Right Kidney, Left Kidney, Spleen, Pancreas',
 '09': 'a computerized tomography of Spleen, Right Kidney,

In [11]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)



torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16


In [35]:
dic = {}
for key,value in template_text.items():
    text_inputs = clip.tokenize(value).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    print(text_features.shape, text_features.dtype)
    dic[key] = text_features.to(dtype=torch.float32)

torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16


In [36]:
with open('embeddings_template.pkl','wb') as file:
    pickle.dump(dic,file)

In [15]:
#now generating mean of the template sentences. 

In [28]:
template_text_mean = {}
for key, value in TEMPLATE.items():
    template_text_mean[key] = generate_sentence_list(value)

In [29]:
template_text_mean

{'01': ['a computerized tomography of Spleen',
  'a computerized tomography of Right Kidney',
  'a computerized tomography of Left Kidney',
  'a computerized tomography of Gall Bladder',
  'a computerized tomography of Esophagus',
  'a computerized tomography of Liver',
  'a computerized tomography of Stomach',
  'a computerized tomography of Arota',
  'a computerized tomography of Postcava',
  'a computerized tomography of Portal Vein and Splenic Vein',
  'a computerized tomography of Pancreas',
  'a computerized tomography of Right Adrenal Gland',
  'a computerized tomography of Left Adrenal Gland',
  'a computerized tomography of Duodenum'],
 '02': ['a computerized tomography of Pancreas'],
 '03': ['a computerized tomography of Liver'],
 '04': ['a computerized tomography of Liver',
  'a computerized tomography of Liver Tumor'],
 '05': ['a computerized tomography of Right Kidney',
  'a computerized tomography of Left Kidney',
  'a computerized tomography of Kidney Tumor',
  'a comput

In [30]:
dic_mean = {} 
for key,values in template_text_mean.items():
    text_inputs = torch.cat([clip.tokenize(value) for value in values]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    mean_text_features = torch.mean(text_features,dim=0).unsqueeze(0)
    print(text_features.shape, text_features.dtype)
    print(mean_text_features.shape, mean_text_features.dtype)
    dic_mean[key] = mean_text_features.to(dtype = torch.float32)

torch.Size([14, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([2, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([4, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([9, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([17, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([5, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([15, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([6, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([13, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([2, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([2, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([1, 512]) torch.float16
torch.Size([2, 5

In [31]:
dic_mean['01'].dtype

torch.float32

In [26]:
dic_mean.keys()

dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '12', '13', '14', '10_03', '10_06', '10_07', '10_08', '10_09', '10_10'])

In [32]:
with open('embeddings_template_mean.pkl','wb') as file:
    pickle.dump(dic_mean,file)